# Build and test the normalized difference container

This notebook is linked to https://eoap.github.io/mastering-app-package/containers/norm-diff/

## Goal

Create a container and run the normalized difference step in the container image.


## Setup the environment

In [1]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [3]:
cat ${WORKSPACE}/water-bodies/command-line-tools/norm_diff/Dockerfile

FROM docker.io/python:3.10-slim

RUN pip install --no-cache-dir rasterio click loguru && \
    python -c "import rasterio"

ADD app.py /app/app.py

ENTRYPOINT []



Build the container using `podman`:

In [7]:
podman build --format docker -t localhost/norm-diff:latest ${WORKSPACE}/water-bodies/command-line-tools/norm_diff


STEP 1/4: FROM docker.io/python:3.10-slim
STEP 2/4: RUN pip install --no-cache-dir rasterio click loguru &&     python -c "import rasterio"
--> Using cache 5242a5830b84d375022f5459fea09295ee02dd76200ca7eba3f5ad08894c69e0
--> 5242a5830b8
STEP 3/4: ADD app.py /app/app.py
--> Using cache 970e5b06c0b248c340c5ecbab6dad6cbbf17b76b8bb651dc5ca1d9bb3db25ba5
--> 970e5b06c0b
STEP 4/4: ENTRYPOINT []
--> Using cache a2cb333a5c463b89520ba64c8f81080bbf6141cb2f13dcb13d3a7a93088c5f09
COMMIT localhost/norm-diff:latest
--> a2cb333a5c4
Successfully tagged localhost/norm-diff:latest
Successfully tagged localhost/norm_diff:latest
a2cb333a5c463b89520ba64c8f81080bbf6141cb2f13dcb13d3a7a93088c5f09


Show the `norm-diff` help:

In [8]:
podman run --rm -it --env=PYTHONPATH=/app localhost/norm-diff:latest python -m app --help

Usage: app.py [OPTIONS] RASTERS...

  Performs a normalized difference

Options:
  --help  Show this message and exit.


## Test the normalized difference step in the container

Generate the normalized difference between the green and nir bands:

In [9]:
podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/crop_green.tif,target=/inputs/crop_green.tif,readonly \
    --mount=type=bind,source=/workspace/mastering-app-package/runs/crop_nir.tif,target=/inputs/crop_nir.tif,readonly \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/norm-diff:latest \
    python \
    -m \
    app \
    /inputs/crop_green.tif \
    /inputs/crop_nir.tif

2024-04-11 13:31:01.641 | INFO     | __main__:normalized_difference:18 - Processing the normalized image with /inputs/crop_green.tif and /inputs/crop_nir.tif
2024-04-11 13:31:03.219 | INFO     | __main__:normalized_difference:39 - Write norm_diff.tif
2024-04-11 13:31:09.840 | INFO     | __main__:normalized_difference:42 - Done!


List the outputs:

In [10]:
tree ${RUNTIME}

/workspace/mastering-app-package/runs
├── crop_green.tif
├── crop_nir.tif
└── norm_diff.tif

0 directories, 3 files
